In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from transformers import RobertaTokenizer

%matplotlib inline

In [2]:
def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer, orig_answer_text):
    """Returns tokenized answer spans that better match the annotated answer."""
    tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))

    for new_start in range(input_start, input_end + 1):
        for new_end in range(input_end, new_start - 1, -1):
            text_span = " ".join(doc_tokens[new_start : (new_end + 1)])
            if text_span == tok_answer_text:
                return (new_start, new_end)

    return (input_start, input_end)

In [3]:
lt_ids = pd.read_pickle('../input/localtest_ids.pkl')

In [4]:
train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
train['type'] = 'normal'

In [5]:
# remove test
print(train.shape)
train = train[~train['textID'].isin(lt_ids['textID'].tolist())]
print(train.shape)

(27481, 5)
(24001, 5)


In [6]:
pseudo = pd.read_pickle('../input/roberta_pred.pkl')
pseudo = pseudo[pseudo['score']>=1.2]
pseudo['selected_text'] = pseudo['pred'].values
pseudo['type'] = 'pseudo'

In [7]:
pseudo['text'] = pseudo['text'].apply(lambda x: ' '.join(x.lower().strip().split()))
pseudo['selected_text'] = pseudo['selected_text'].apply(lambda x: ' '.join(x.lower().strip().split()))

In [8]:
pseudo.drop(pseudo[pseudo['selected_text']==''].index, inplace=True)

In [9]:
tokenizer = RobertaTokenizer.from_pretrained('../../bert_models/roberta_base/')

In [10]:
train.shape

(24001, 5)

In [11]:
train.dropna(subset=['text','selected_text'], how='any', inplace=True)

In [12]:
train.shape

(24000, 5)

In [13]:
train.head()

,textID,text,selected_text,sentiment,type
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,normal
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,normal
3,9642c003ef,what interview! leave me alone,leave me alone,negative,normal
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,normal
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral,normal


In [14]:
train['text'] = train['text'].apply(lambda x: ' '.join(x.lower().strip().split()))
train['selected_text'] = train['selected_text'].apply(lambda x: ' '.join(x.lower().strip().split()))

In [15]:
def clean(x):
    sp_x = x.split()
    if sp_x[0] in [',','.','!','?',':',';'] and len(sp_x)>1:
#         print(x, '|', ' '.join(sp_x[1:]))
        return ' '.join(sp_x[1:])
    return x

In [16]:
train['selected_text'] = train['selected_text'].apply(lambda x: clean(x))

In [17]:
# 拼接
train = pd.concat([train, pseudo], ignore_index=True, sort=False)
train.reset_index(inplace=True, drop=True)

In [18]:
train['start_pos'] = train.apply(lambda x: x['text'].find(x['selected_text']), axis=1)
train['end_pos'] = train.apply(lambda x: x['start_pos']+len(x['selected_text']), axis=1)

In [19]:
def contains(a, b):
    for i in range(0, len(a)-len(b)+1):
        flag = True
        for j in range(0, len(b)):
            if a[i+j]!=b[j]:
                flag = False
                break
        if flag:
            return True, i
    return False, -1

In [20]:
data = []
improve_count = 0
for text, sp, ep, st in zip(train['text'].tolist(), train['start_pos'].tolist(), train['end_pos'].tolist(),
                           train['selected_text'].tolist()):
    split_text = text.split()
    tokens, labels, invert_map, first_token, in_st = [], [], [], [], []
    st_tokens = tokenizer.tokenize(' '+st)
    cur_length = 0
    
    # token in selected_tokens
    temp = np.zeros(len(text))
    temp[sp:ep]=1
    for idx, w in enumerate(split_text):
        if sum(temp[cur_length+idx:cur_length+idx+len(w)])>0:
            started = True# space
        else:
            started = False
        for idx2, token in enumerate(tokenizer.tokenize(' '+w)):
            first_token.append(True if idx2==0 else False)
            tokens.append(token)
            invert_map.append(idx)
            if started:
                labels.append(len(tokens)-1)
                if token in st_tokens:
                    in_st.append(1)
                else:
                    in_st.append(0)
            else:
                in_st.append(-100)
        cur_length+=len(w)
    start_token_idx = min(labels)
    end_token_idx = max(labels)
    start_token_idx, end_token_idx = _improve_answer_span(tokens, start_token_idx, end_token_idx,
                                                         tokenizer, ' '+st)
    
#     token_contain, token_sp = contains(tokens, st_tokens)
#     if token_contain:
#         start_token_idx = token_sp
#         end_token_idx = token_sp+len(st_tokens)-1
    
    start_word_idx = invert_map[start_token_idx]
    end_word_idx = invert_map[end_token_idx]+1
    assert ' '.join(split_text[start_word_idx:end_word_idx]).lower().find(st.lower())>=0
    
    data.append((tokens, start_token_idx, end_token_idx, invert_map, first_token, in_st))

In [21]:
tokens, start, end, invert_map, first_token, in_st = zip(*data)

In [22]:
len(tokens)

26610

In [23]:
train['tokens'] = tokens

In [24]:
train['start'] = start
train['end'] = end
train['invert_map'] = invert_map
train['first_token'] = first_token
train['in_st'] = in_st

In [25]:
senti2label = {
    'positive':2,
    'negative':0,
    'neutral':1
}
train['senti_label']=train['sentiment'].apply(lambda x: senti2label[x])

In [26]:
train.reset_index(drop=True, inplace=True)

In [27]:
train.head()

,textID,text,selected_text,sentiment,type,start_pos,end_pos,tokens,start,end,invert_map,first_token,in_st,senti_label,pred,score
0,cb774db0d1,"i`d have responded, if i were going","i`d have responded, if i were going",neutral,normal,0,35,"[Ġi, `, d, Ġhave, Ġresponded, ,, Ġif, Ġi, Ġwer...",0,9,"[0, 0, 0, 1, 2, 2, 3, 4, 5, 6]","[True, False, False, True, True, False, True, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,NaN,NaN
1,549e992a42,sooo sad i will miss you here in san diego!!!,sooo sad,negative,normal,0,8,"[Ġso, oo, Ġsad, Ġi, Ġwill, Ġmiss, Ġyou, Ġhere,...",0,2,"[0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9]","[True, False, True, True, True, True, True, Tr...","[1, 1, 1, -100, -100, -100, -100, -100, -100, ...",0,NaN,NaN
2,9642c003ef,what interview! leave me alone,leave me alone,negative,normal,16,30,"[Ġwhat, Ġinterview, !, Ġleave, Ġme, Ġalone]",3,5,"[0, 1, 1, 2, 3, 4]","[True, True, False, True, True, True]","[-100, -100, -100, 1, 1, 1]",0,NaN,NaN
3,358bd9e861,"sons of ****, why couldn`t they put them on th...","sons of ****,",negative,normal,0,13,"[Ġsons, Ġof, Ġ****, ,, Ġwhy, Ġcouldn, `, t, Ġt...",0,3,"[0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 10, 11...","[True, True, True, False, True, True, False, F...","[1, 1, 1, 1, -100, -100, -100, -100, -100, -10...",0,NaN,NaN
4,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral,normal,0,92,"[Ġhttp, ://, www, ., d, othe, b, oun, cy, ., c...",0,26,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[True, False, False, False, False, False, Fals...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,NaN,NaN


In [28]:
from sklearn.model_selection import StratifiedKFold

In [29]:
train.shape

(26610, 16)

In [40]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

In [41]:
train['fold'] = 0
for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train['senti_label'])):
    train.loc[valid_idx, 'fold'] = fold

In [42]:
train['fold'].value_counts()

0    5323
3    5322
2    5322
1    5322
4    5321
Name: fold, dtype: int64

In [45]:
train.to_pickle('../input/train_roberta_with_pseudo.pkl')

In [43]:
train['type'].value_counts()

normal    24000
pseudo     2610
Name: type, dtype: int64

In [44]:
train[train['fold']==1]['type'].value_counts()

normal    4791
pseudo     531
Name: type, dtype: int64